## Setup

In [1]:
import torch
import numpy as np
from torch import einsum
from tqdm.auto import tqdm
import seaborn as sns
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from datasets import load_dataset
from einops import einsum
import pandas as pd
from transformer_lens import utils
from rich.table import Table, Column
from rich import print as rprint
from jaxtyping import Float, Int, Bool
from typing import List, Tuple
from torch import Tensor
import einops
import functools
from transformer_lens.hook_points import HookPoint
# import circuitsvis
from IPython.display import HTML
from plotly.express import line
import plotly.express as px
from tqdm.auto import tqdm
import json
import gc
import plotly.graph_objects as go

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotly.subplots import make_subplots
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import load_txt_data, get_mlp_activations, line
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [2]:
english_neurons = [(5, 395), (5, 166), (5, 908), (5, 285), (3, 862), (5, 73), (4, 896), (5, 348), (5, 297), (3, 1204)]
german_neurons = [(4, 482), (5, 1039), (5, 407), (5, 1516), (5, 1336), (4, 326), (5, 250), (3, 669)]
french_neurons = [(5, 112), (4, 1080), (5, 1293), (5, 455), (5, 5), (5, 1901), (5, 486), (4, 975)]

model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

english_data = haystack_utils.load_txt_data("kde4_english.txt")
german_data = haystack_utils.load_txt_data("wmt_german_large.txt")

english_activations = {}
german_activations = {}
for layer in range(3, 6):
    english_activations[layer] = get_mlp_activations(english_data[:200], layer, model, mean=False)
    german_activations[layer] = get_mlp_activations(german_data[:200], layer, model, mean=False)

LOG_PROB_THRESHOLD = -7
LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
MEAN_ACTIVATION_ACTIVE = german_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', deactivate_neurons_hook)]

    
def activate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_ACTIVE
    return value
activate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', activate_neurons_hook)]

all_ignore, not_ignore = haystack_utils.get_weird_tokens(model, plot_norms=False)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
kde4_english.txt: Loaded 1007 examples with 501 to 5295 characters each.
wmt_german_large.txt: Loaded 2459 examples with 800 to 2000 characters each.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [3]:
def get_pos_loss_diff(prompt: str, model: HookedTransformer, activate_neurons_hook: List[Tuple[str, HookPoint]], deactivate_neurons_hook: List[Tuple[str, HookPoint]], plot_hist=False, use_activate_hook=False, debug_log=True):
    tokens = model.to_tokens(prompt)
    if use_activate_hook:
        original_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=activate_neurons_hook, loss_per_token=True)
    else:
        original_loss = model(tokens, return_type="loss", loss_per_token=True)
    ablated_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=deactivate_neurons_hook, loss_per_token=True)
    
    # Positive difference = loss increase due to ablation
    loss_difference = (ablated_loss - original_loss).flatten()

    if debug_log:
        print(f"Unablated loss: {original_loss.flatten()}")
        print(f"Ablated loss: {ablated_loss.flatten()}")
        print(f"Loss difference: {loss_difference}")

    if plot_hist:
        fig = px.histogram(loss_difference.flatten().cpu().numpy(), title="Loss difference due to ablation per position")
        fig.show()
    return loss_difference

def get_high_loss_prompts(prompts: list[str], model: HookedTransformer, activate_neurons_hook: List[Tuple[str, HookPoint]], deactivate_neurons_hook: List[Tuple[str, HookPoint]]):
    max_diffs = []
    average_diffs = []
    for prompt in tqdm(prompts):
        loss_difference = get_pos_loss_diff(prompt, model, activate_neurons_hook, deactivate_neurons_hook)
        max_diffs.append(loss_difference.max().item())
        average_diffs.append(loss_difference.mean().item())
    return max_diffs, average_diffs


In [4]:
def get_top_differences_at_position(prompt: str, model: HookedTransformer, position: int, top_k=20, mode="full"):
    tokens = model.to_tokens(prompt)
    str_tokens = model.to_str_tokens(tokens)
    # Logprobs instead of logits
    original_logits = model(tokens, return_type="logits")
    if mode=="direct":
        to_freeze = ["blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"]
        ablated_logits = haystack_utils.get_frozen_logits(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=to_freeze)
    elif mode=="indirect":
        to_freeze = ["blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"]
        ablated_logits = haystack_utils.get_ablated_logits(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=to_freeze)
    else:
        assert mode=="full"
        ablated_logits = model.run_with_hooks(tokens, return_type="logits", fwd_hooks=deactivate_neurons_fwd_hooks)
    original_logprob = original_logits.log_softmax(dim=-1)
    ablated_logprob = ablated_logits.log_softmax(dim=-1)

    # Positive difference = the German neuron makes the token more likely
    # Negative difference = the German neuron makes the token less likely
    logprob_differences = original_logprob - ablated_logprob
    logit_differences = original_logits - ablated_logits

    print("Prompt:", prompt)
    print(f"Differences for predicting: {str_tokens[position]} -> {str_tokens[position+1]}")

    low_log_prob = torch.argwhere(((original_logprob[0, position, :] <= LOG_PROB_THRESHOLD) & (ablated_logprob[0, position, :] <= LOG_PROB_THRESHOLD))).flatten()
    ignore_tokens = torch.cat([low_log_prob, all_ignore]).unique()
    
    top_original_logprobs, top_original_idx = haystack_utils.top_k_with_exclude(original_logprob[0, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_original_ablated_logprobs = ablated_logprob[0, position, top_original_idx]
    top_ablated_logprobs, top_ablated_idx = haystack_utils.top_k_with_exclude(ablated_logprob[0, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_ablated_original_logprobs = original_logprob[0, position, top_ablated_idx]

    top_original_values = logprob_differences[0, position, top_original_idx]
    top_ablated_values = logprob_differences[0, position, top_ablated_idx]
    top_original_logit_diff = logit_differences[0, position, top_original_idx]
    top_ablated_logit_diff = logit_differences[0, position, top_ablated_idx]
    print("Top predictions with German neuron active (unablated)")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_original_idx), top_original_values.cpu().tolist(), max_value=5, original_log_probs=top_original_logprobs.cpu().tolist(), ablated_log_probs=top_original_ablated_logprobs.cpu().tolist(), logit_difference=top_original_logit_diff.cpu().tolist())
    print("Top predictions with German neuron disabled (ablated)")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_ablated_idx), top_ablated_values.cpu().tolist(), max_value=5, original_log_probs=top_ablated_original_logprobs.cpu().tolist(), ablated_log_probs=top_ablated_logprobs.cpu().tolist(), logit_difference=top_ablated_logit_diff.cpu().tolist())

    top_boosts, top_boosted_idx = haystack_utils.top_k_with_exclude(logprob_differences[:, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_boost_original_logprob = original_logprob[0, position, top_boosted_idx]
    top_boost_ablated_logprob = ablated_logprob[0, position, top_boosted_idx]
    top_reduced, top_reduced_idx = haystack_utils.top_k_with_exclude(logprob_differences[:, position, :].flatten(), top_k, largest=False, exclude=ignore_tokens)
    top_reduced_original_logprob = original_logprob[0, position, top_reduced_idx]
    top_reduced_ablated_logprob = ablated_logprob[0, position, top_reduced_idx]
    print("Top boosted tokens by German neuron")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_boosted_idx), top_boosts.cpu().tolist(), max_value=5, original_log_probs=top_boost_original_logprob.cpu().tolist(), ablated_log_probs=top_boost_ablated_logprob.cpu().tolist())
    print("Top reduced tokens by German neuron")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_reduced_idx), top_reduced.cpu().tolist(), max_value=5, original_log_probs=top_reduced_original_logprob.cpu().tolist(), ablated_log_probs=top_reduced_ablated_logprob.cpu().tolist())

In [15]:
def show_token_loss(prompt: str, model: HookedTransformer, max_value=None, mode="full", freeze_act_names=("blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"), custom_loss_change=None):
    
    original_loss, total_effect_loss_change, direct_effect_loss_change, indirect_effect_loss_change = haystack_utils.split_effects(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names, debug_log=False)
    if mode== "indirect":
        pos_wise_loss = indirect_effect_loss_change
        #pos_wise_loss = haystack_utils.get_frozen_loss_difference_measure(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names)
    elif mode == "direct":
        pos_wise_loss = direct_effect_loss_change
        #pos_wise_loss = haystack_utils.get_ablated_loss_difference_measure(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names)
    elif mode == "custom":
        pos_wise_loss = custom_loss_change
    else:
        assert mode =="full"
        pos_wise_loss = total_effect_loss_change
        #pos_wise_loss = get_pos_loss_diff(prompt, model, activate_neurons_fwd_hooks, deactivate_neurons_fwd_hooks, plot_hist=False)
    str_token_prompt = model.to_str_tokens(model.to_tokens(prompt))
    haystack_utils.print_strings_as_html(str_token_prompt[1:], pos_wise_loss.flatten().cpu().tolist(), max_value=max_value)

def print_predictions(prompt, pos, k=20):
    print("\nFull model predictions")
    get_top_differences_at_position(prompt, model, pos, k, mode="full")
    print("\nIndirect predictions (leave German neuron active, patch corrupted activations to later components)")
    get_top_differences_at_position(prompt, model, pos, k, mode="indirect")
    print("\nDirect predictions (ablate German neuron, patch clean activations to later components)")
    get_top_differences_at_position(prompt, model, pos, k, mode="direct")

## Look for interesting examples with new loss difference sorting

In [33]:
def custom_diff_weight(total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss):
    # We want high loss difference between total effect and direct effect of the context neuron (so restoring MLP4 and MLP5 fixes the problem)
    indirect_diff = total_effect_loss - direct_mlp3_loss
    # We want high loss difference between frozen MLP4 and the direct effect of the context neuron (so only restoring MLP4 is not enough)
    mlp_4_diff = direct_mlp3_mlp5_loss - direct_mlp3_loss
    # We want high loss difference between frozen MLP5 and the direct effect of the context neuron (so only restoring MLP5 is not enough)
    mlp_5_diff = frozen_loss - direct_mlp3_loss
    return indirect_diff, mlp_4_diff, mlp_5_diff

In [84]:
def get_interesting_loss_prompts(prompts: list[str], model: HookedTransformer):
    max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff = [], [], []
    for prompt in tqdm(prompts):
        original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=100, log=False, answer_pos=None)
        indirect_diff, mlp_4_diff, mlp_5_diff = custom_diff_weight(total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss)
        max_indirect_diff.append(indirect_diff.max().item())
        max_mlp_4_diff.append(mlp_4_diff.max().item())
        max_mlp_5_diff.append(mlp_5_diff.max().item())
    return max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff

max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff = get_interesting_loss_prompts(german_data, model)

  0%|          | 0/2459 [00:00<?, ?it/s]

In [85]:
# Filter for examples with high difference in top MLP5 neurons and MLP4+MLP5 - these are the examples where patching some MLP5 is not enough (so maybe they rely on MLP4)
loss_data_tuple = [(diff, example) for diff, example in zip(max_mlp_5_diff, german_data[:len(max_mlp_5_diff)])]
loss_data_tuple.sort(key=lambda x: x[0], reverse=True)
loss_data_tuple[:2]

[(2.4673376083374023,
  'Zur Zielsetzung, die das Vereinigte Königreich verfolgt - die völlige Aufhebung des Embargos - möchte ich unterstreichen, daß der Rat in seinen Schlußfolgerungen vom 29. und 30. April klar gesagt hat - und hier beziehe ich mich auf Absatz 6 dieser Schlußfolgerungen -, daß die Gesamtheit der vom Vereinigten Königreich ergriffenen Maßnahmen, deren Umsetzung sowie Kontrolle durch die Kommission, nämlich das von den britischen Behörden angekündigte selektive Schlachtprogramm, die sich als notwendig erwiesenen Zusatzmaßnahmen sowie schließlich die ebenso notwendige Bekräftigung, sich bei zukünftigen Beschlußfassungen immer mehr auf solide wissenschaftliche Aussagen zu stützen, Anhaltspunkte darstellen, die zu dem Prozeß gehören, der eine allmähliche Aufhebung des Ausfuhrverbots ermöglichen dürfte.'),
 (2.2395410537719727,
  'Der Europäische Rat stellte fest, dass weitere Schritte in Richtung einer EU-Mitgliedschaft erstens auf der Grundlage der Debatte über die Erwe

In [118]:
for _ , prompt in loss_data_tuple[100:110]:
    print(f"\n{prompt}")
    original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=50, log=False, answer_pos=None)    
    mlp_5_diff = frozen_loss - direct_mlp3_loss
    mlp_4_diff = direct_mlp3_mlp5_loss - direct_mlp3_loss
    show_token_loss(prompt, model, max_value=5, mode="custom", custom_loss_change=mlp_4_diff)
    show_token_loss(prompt, model, max_value=5, mode="custom", custom_loss_change=mlp_5_diff)
    # Scan for examples with reasonable overall loss
    show_token_loss(prompt, model, max_value=10, mode="custom", custom_loss_change=original_loss*-1)
    #show_token_loss(prompt, model, max_value=5, mode="full")
    #show_token_loss(prompt, model, max_value=5, mode="indirect")
    #show_token_loss(prompt, model, max_value=5, mode="direct")


Abschließend möchte ich also sagen, wir sollten vorbehaltlich folgender Forderungen - 1. Vorlage eines kompletten Informationsdossiers vor jeder Marktzulassung, 2. Einhaltung einer strikten Kennzeichnungspflicht, 3. Rückverfolgbarkeit, Überwachung und detaillierte Bewertung der Vorteile und Risiken sowie Vorsichtsmaßnahmen in jedem Stadium, Haftung der Hersteller für alle Schäden an Personen und Gütern und selbstverständlich sofortige Einstellung der Vermarktung im Alarmfalle - die Züchtung von GVO zum Zwecke ihrer Vermarktung innerhalb der Europäischen Union zulassen, um nicht unsere Mitgliedstaaten zu benachteiligen und bestimmte Länder wie die USA zu bevorteilen, die genetisch veränderte Erzeugnisse herstellen und verbrauchen und deren Produkte mehr und mehr unseren Gemeinschaftsmarkt erobern und noch weiter erobern werden.



Herr Poettering, Sie sprechen zu Recht von der Notwendigkeit, den Terrorismus bei der Wurzel zu packen, und aus meiner Sicht wäre es dringend erforderlich, dass die Europäische Union sich mit ihrer Politik beschäftigt, mit der Politik der letzten 50 Jahre, die gegenüber der arabischen und islamischen Welt zutiefst rassistisch war, eine Politik, die nur die schlimmsten Regimes in diesen Ländern unterstützt hat, das von Saddam Hussein, von Gaddafi, von Hafez El-Assad gestern und seinem Sohn heute, und kaum Solidarität mit den Demokraten von gestern und heute geübt hat, wie Frau Ben Sedrin in Tunesien, wie Frau Al-Sadaoui in Ägypten, wie Herr Ibrahimi in Ägypten und so viele andere, die für die Demokratie in ihren Ländern kämpfen. Ich halte es für dringend erforderlich, dass unsere Union die Herstellung der Demokratie in diesen Ländern zu ihrer politischen Priorität erhebt, denn dies ist für mich der einzige Weg, um die Rückzugsbasen des Terrorismus in diesen Ländern zu zerschlagen und z


Die Mitglieder der Europäischen Kommission haben sich auch bemüht, dies in den Anhörungen in den Ausschüssen des Europäischen Parlaments zu bekräftigen, indem sie ohne zu zögern folgende Punkte unterstützten: die imperialistische Ausrichtung der EU, die Stärke ihrer volksfeindlichen Politik und der Militärinterventionen auf internationaler Ebene, um die Profite des Kapitals zu erhöhen; ihr Einsatz für die Vollendung des Binnenmarktes und der vier Freiheiten des Maastrichter Vertrages, für Wettbewerbsfähigkeit und kapitalistische Umstrukturierungen, um günstige Bedingungen für die Konzentration des Kapitals zu schaffen; das Fördern von volksfeindlichen Maßnahmen durch umfassende Angriffe auf die Stellen, das Gehalt sowie soziale und Versicherungsrechte der Arbeiter, um die Profite der Monopolisten zu steigern; eine Erweiterung und Stärkung von reaktionären Bestrafungsinstrumenten, um Demonstrationen der Arbeiter und des Volkes zu unterdrücken.



Natürlich ist dies eine Richtlinie, die nicht von einer der vorhergehenden britischen Regierungen umgesetzt wurde, sondern von einer darauf folgenden Regierung, nämlich der jetzigen. Die Kommission leitete ein Vertragsverletzungsverfahren nach Artikel 226 ein, jedoch änderte sich die Situation, die Richtlinie wurde in nationales Recht umgesetzt und, wie Kommissar McCreevy erklärte, konnte das Verfahren nach der Rechtsprechung des Gerichtshofs der Europäischen Union von jenem Zeitpunkt an vor dem Gerichtshof nicht mehr fortgesetzt werden, denn Artikel 226 lautet: „Hat nach Auffassung der Kommission ein Mitgliedstaat gegen eine Verpflichtung aus diesem Vertrag verstoßen, so gibt sie eine mit Gründen versehene Stellungnahme hierzu ab; sie hat dem Staat zuvor Gelegenheit zur Äußerung zu geben“.



Il patrimonio genetico di questa razza ,che si è mantenuto incontaminato in epoca moderna anche grazie al concentramento della popolazione presso l’allevamento di un’unica famiglia di Laconi,Der Genpool dieser Rasse, die unberührt geblieben ist in der heutigen Zeit, dank der Konzentration der Bevölkerung bei der Zucht einer einzigen Familie , der Familie Laconi, è una vera e propria miniera d’informazioni che potrebbe fornire dati determinanti per gli studi sull’evoluzione del cavallo stesso.ist eine echte Fundgrube für Informationen, die wichtige Daten für Studien über die Entwicklung des Pferdes geben. Lo studio del Dna effettuato, prova l’estraneità del cavallo del Sarcidano rispetto alle altre popolazioni equine italiane, confermando le antichissime origini ,Die Studie der DNA, untersucht die Fremdheit der Sarcidano Pferdes im Vergleich zu anderen Pferden in Italien, was fuer die antiken Ursprünge risulta quindi importante ai fini della preservazione della biodiversità delle razze


A4-0058/96 von Frau Kjer Hansen im Namen des Ausschusses für Haushaltskontrolle über die Entlastung der Kommission für den Rechnungsabschluß des EAGFL für das Haushaltsjahr 1991 (LET 11877 - C4-0591/95); -A4-0086/91 von Herrn Cornelissen im Namen des Ausschusses für Haushaltskontrolle über den Bericht der Kommission über die Maßnahmen im Anschluß an die Bemerkungen in der Entschließung zum Beschluß über die Entlastung zur Ausführung des Gesamthaushaltsplans der Europäischen Gemeinschaften für das Haushaltsjahr 1992 (KOM(95)0666); -A4-0097/91 von Herrn Samland im Namen des Ausschusses für Haushaltskontrolle über den Bericht der Kommission über die Maßnahmen im Anschluß an die Bemerkungen in der Entschließung zum Beschluß über die Entlastung zur Ausführung des Gesamthaushaltsplans der Europäischen Gemeinschaften für das Haushaltsjahr 1993 (KOM(95)0666); -A4-0098/96 von Herrn Wynn im Namen des Ausschusses für Haushaltskontrolle über die Entlastung der Kommission zur Ausführung des Gesamt


Die, teilweise noch aus dem Beginn des 19. Jahrhunderts stammenden, Rechtsinstitute des Österreichischen Zivilrechtes auf Rechtsgeschäfte des 21. Jahrhunderts - Stichwort: e-commerce - anzuwenden, ist eine der spannendsten Herausforderungen für den Rechtsanwender unserer Zeit.Nur, wer neben einer profunden rechtlichen Ausbildung und Praxis auch über das nötige technische Verständnis verfügt, wird diese Herausforderung meistern.Lambert Eversheds vertritt in- und ausländische Telekommunikationsunternehmen, Internet-Provider und Software-Hersteller. Egal, ob es sich um Lizenzierungsverfahren vor der Regulierungsbehörde, innovative Vertragsgestaltungen über Telekommunikationsleitungen oder um Urheberrechtsstreitigkeiten auf dem Software-Markt handelt - wir haben dazu kreative Ideen und die nötige Erfahrung.Ein weiterer Schwerpunkt der diesbezüglichen Tätigkeit von Lambert Eversheds ist der, in seiner Bedeutung heute noch oftmals unterschätzte, Bereich des Datenschutzes.



Wie auch der Europäische Wirtschafts- und Sozialausschuss unterstützt das Europäische Parlament das Ziel der Kommission, das sie mit der Einbringung dieser Initiative verfolgt und das darin besteht, Kleinstunternehmen von den Anforderungen hinsichtlich Verwaltung und Rechnungslegung zu befreien, die in Bezug auf den Bedarf und die internen Unternehmensstrukturen von Kleinstunternehmen und den Hauptnutzern von Finanzinformationen kostspielig und völlig unverhältnismäßig sind. Sie sollen somit in die Lage versetzt werden, die zahlreichen strukturellen Herausforderungen, die einem komplexen Unternehmen innewohnen, dank der vollständigen Implementierung der Europäischen Charta für Kleinstunternehmen und entsprechend eines in die Strategie von Lissabon integriertes Verfahren bewältigen zu können.



Das Leben wird als eine Reise gesehen, die von göttlichem Gesetz geregelt wird, nach dem sich der Mensch richten muß, wenn er Ruin zu vermeiden wünscht; es wird gesehen als jenes rechte Ordnen von öffentlichem und privatem Leben, das wir in der Almanach-Literatur finden, im Pastoralen und in der Legende einer der Industrialisierung vorausgehenden Ära der deutschen Gesellschaft des 18. Jahrhunderts mit ihren aufwendigen Höfen, arrogantem Adel, schüchterner Mittelklasse und einem Bauernstand, unfähig, sich auszudrücken (WR n° 17); darauf folgt eine Art Revolte gegen diese Ordnung, eine Suche nach Freiheit vom Zeitalter der Aufklärung, hin zum Zeitalter des Verstehens, das eine neue Welt eröffnete, in der das kreative Volk der Held-oder Anti-Held- werden sollte, während die intellektuelle Hegemonie Europas um die Jahrhundertwende von Frankreich auf Deutschland überging.



Die von uns eingebrachten Änderungsanträge, zu denen wir die Meinung der Kommission erfahren möchten, und die der Rat hoffentlich bei seiner Stellungnahme als entschlossenen Willen des Europäischen Parlaments betrachten wird, zielen insbesondere darauf ab, die Modalitäten und Definitionen vonInformation und Anhörung genauer festzulegen - vor allem was den Zeitpunkt betrifft, zu dem die Informationen übermittelt werden müssen, nämlich in der Planungsphase, damit sich die Arbeitnehmer rechtzeitig auf die künftigen Entwicklungen einstellen können, aber auch was den Inhalt der Information betrifft, der auf andere Bereiche ausgeweitet werden muß, die in direktem Zusammenhang mit den Rechten der Arbeitnehmer und ihrer Zukunft stehen. Ein weiteres Ziel ist, den Anwendungsbereich der Richtlinie auszuweiten; hierzu schlagen wir vor, ihn auf den öffentlichen Dienst auszudehnen und die Mitgliedstaaten zu verpflichten, über Möglichkeiten der Einbeziehung kleiner und mittlerer Unternehmen nachzude

In [120]:
prompt = 'Abschließend möchte ich also sagen, wir sollten vorbehaltlich folgender Forderungen - 1. Vorlage eines kompletten Informationsdossiers vor jeder Marktzulassung'
k = 200
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

## The only actual example where both matter I found actually k=500 fixes it :(

In [115]:
prompt = 'Bericht (A4-0220/97) von Frau Pery im Namen des Ausschusses für Fischerei über den Vorschlag für einen Beschluß des Rates über den Abschluß des Abkommens in Form eines Briefwechsels zur vorübergehenden Verlängerung des Protokolls zum Abkommen zwischen der Europäischen Gemeinschaft und der Regierung der Republik Senegal über die Fischerei vor der senegalesischen Küste vom 2. Oktober 1996 bis 1. November 1996 (KOM(96)0611 - C4-0032/97-96/0287(CNS)); -Bericht (A4-0224/97) von Frau Péry im Namen des Ausschusses für Fischerei über den Vorschlag für eine Verordnung des Rates über den Abschluß des Protokolls zur Festlegung der Fischereirechte und des finanziellen Ausgleichs nach dem Abkommen zwischen der Regierung der Republik Senegal und der Europäischen Gemeinschaft über die Fischerei vor der senegalesischen Küste für die Zeit vom 1. Mai 1997 bis zum 30. April 2001 (KOM(97)0324 - C4-0322/97-97/0179(CNS)); -Bericht (A4-0229/97) von Herrn Gallagher über den Vorschlag für eine Verordnung (EG) des Rates über den Abschluß des Pro'
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

## Something cool? MLP4 doing things when "." is continued with lowercase word
- On "und" between dates ("12. und 13. Dezember")
- After "usw" (=etc) ("... usw. zu fragen")
- Unfortunately the model is very bad at this in general (except for first example)

In [91]:
prompt = 'Ein einzigartiges Erlebnis ist es, mit staunendem Herzen Hand in Hand das Feuerwerk vom 14. Juli von der Rondinara Bucht aus zu verfolgen, gemeinsam über die Strände von Santa Giulia zu'
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [61]:
prompt = 'Zur Zielsetzung, die das Vereinigte Königreich verfolgt - die völlige Aufhebung des Embargos - möchte ich unterstreichen, daß der Rat in seinen Schlußfolgerungen vom 29. und'
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [58]:
prompt = 'Der Europäische Rat stellte fest, dass weitere Schritte in Richtung einer EU-Mitgliedschaft erstens auf der Grundlage der Debatte über die Erweiterungsstrategie gemäß den Schlussfolgerungen des Rates vom Dezember 2005 geprüft werden müssten, die in den "erneuerten Konsens" über die Erweiterung, welcher auf dem Europäischen Rat am 14. und'
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [59]:
prompt = 'Blicken wir auf die politisch-soziale Situation, in der die eingangs wiedergegebene Frage, deren Formulierung ich einem Buch des in Buenos Aires tätigen Colectivo Situaciones entnehme, ebenso wie ihre vorläufig nur knapp umrissenen Implikationen ihre konkrete Verortung finden: Sie betrifft die insbesondere an den Tagen des 19. und'

k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [60]:
prompt = 'Deshalb ist es falsch, den Schutz der biologischen Vielfalt strikt auf den zweiten Pfeiler der gemeinsamen Agrarpolitik - und insbesondere auf die Agrarumweltprogramme zur Förderung der Extensivierung, des ökologischen Landbaus, zur Erhaltung einheimischer Rassen, zum Schutz der natürlichen Lebensräume usw. zu'
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

## Clean examples of Layer 5 doing things

In [102]:
prompt = 'Zum Abschluss erlauben Sie mir bitte, all jenen Abgeordneten zu danken, die anwesend sind und denen, die an dieser Aussprache teilnehmen werden, aber wegen des Vulkanausbruchs nicht hier sein können, die viel Arbeit geleistet haben, um den Vorschlag für eine Entscheidung vorzubere'
k = 20
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

## Superposition examples
- The model needs more MLP5 neurons to restore the loss

In [121]:
prompt = 'Abschließend möchte ich also sagen, wir sollten vorbehaltlich folgender Forderungen - 1. Vorlage eines kompletten Informationsdossiers vor jeder Marktzulassung'
k = 200
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [43]:
prompt = "Der Vorschlag zur Änderung dieser Verordnung über die Anwendung des dem Vertrag"
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [70]:
prompt = "Ich möchte nochmals meine Ansicht"
k = 20
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [77]:
prompt = "   .– Herr Präsident, als Vorsitzender des Ausschusses für Recht und Binnenmarkt sei mir die Behauptung gestattet, dass ich niemals so stolz wie heute Abend war, Vorsitzender eines Ausschusses zu sein, der dem Ausschuss für die Freiheiten und Rechte der Bürger, Justiz und innere Angelegenheiten eine objektive und sehr umfassende Stellungnahme sowie detaillierte Vorschläge vorgelegt hat, in denen er äußerst bewusst und vollkommen sachlich, ohne Widerspruch und sehr rigoros ein Bemühen um Diversifizierung des Eigentums und der Kontrolle des Fernsehens"
k = 20
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

In [83]:
prompt = 'Deshalb ist es für uns als Konföderale Fraktion der Vereinigten Europäischen Linken/Nordische Grüne Linke von grundlegender Bedeutung, Kommission und Rat mit der vom Parlament anzunehmenden Entschließung'
k = 20
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")